In [185]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [186]:
def scale(df):
    scaler = StandardScaler()
    scaler.fit(df)
    #print(scaler.mean_)
    return scaler.transform(df)

In [187]:
def encode_values(df):
    le = preprocessing.LabelEncoder()
    le.fit(df)
    print(len(le.classes_))
    return le.transform(df)

In [188]:
def preprocess(dataset):
    for col in ['Geography']:
        dataset[col] = encode_values(dataset[col].values)
    dataset = dataset.drop(['CustomerId'],axis=1)
    dataset = dataset.drop(['Surname'],axis=1)
    dataset = dataset.drop(['Gender'],axis=1)
    temp_dataset = scale(dataset.drop('Exited',axis=1).values)
    new_dataset = pd.DataFrame(temp_dataset,columns=['CreditScore', 'Geography', 'Age', 'Tenure', 'Balance',
           'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])
    new_dataset['Exited'] = dataset['Exited']
    sample = new_dataset.sample(frac=1)
    return sample

In [189]:
import numpy as np
np.random.seed(1234)
np.seterr(over='ignore')

class NeuralNetwork():
    def __init__(self):
        np.random.seed(1)  # Seed the random number generator
        self.weights = {}  # Create dict to hold weights
        self.num_layers = 1  # Set initial number of layer to one (input layer)
        self.adjustments = {}  # Create dict to hold adjustements

    def add_layer(self, shape):
        # Create weights with shape specified + biases
        self.weights[self.num_layers] = np.vstack((2 * np.random.random(shape) - 1, 2 * np.random.random((1, shape[1])) - 1))
        # Initialize the adjustements for these weights to zero
        self.adjustments[self.num_layers] = np.zeros(shape)
        self.num_layers += 1

    def __sigmoid(self, x):
        x = np.clip(x,-20,20)
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, data):
        # Pass data through pretrained network
        for layer in range(2, self.num_layers+1):
            #print(self.weights[layer-1][:, :-1])
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][ -1,:].T # + self.biases[layer]
            data = self.__sigmoid(data).T
        return data

    def forward_propagate(self, data):
        # Progapagate through network and hold values for use in back-propagation
        activation_values = {}
        activation_values[1] = data
        for layer in range(2, self.num_layers+1):
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][-1, :].T # + self.biases[layer]
            data = self.__sigmoid(data).T
            activation_values[layer] = data
        return activation_values

    def simple_error(self, outputs, targets):
        return outputs-targets

    def sum_squared_error(self, outputs, targets):
        return 0.5 * np.mean(np.sum(np.power(outputs - targets, 2), axis=1))
    
    def entropy(self,outputs,targets):
        return -(1.0/len(targets)) * np.sum(targets*np.log(outputs) + (1-targets)*np.log(1-outputs))

    def __back_propagate(self, output, target):
        deltas = {}
        # Delta of output Layer
        deltas[self.num_layers] = self.simple_error(output[self.num_layers],target)

        # Delta of hidden Layers
        for layer in reversed(range(2, self.num_layers)):  # All layers except input/output
            a_val = output[layer]
            weights = self.weights[layer][:-1, :]
            prev_deltas = deltas[layer+1]
            deltas[layer] = np.multiply(np.dot(weights, prev_deltas), self.__sigmoid_derivative(a_val))

        # Caclculate total adjustements based on deltas
        for layer in range(1, self.num_layers):
            self.adjustments[layer] += np.dot(deltas[layer+1], output[layer].T).T

    def __gradient_descente(self, batch_size, learning_rate):
        # Calculate partial derivative and take a step in that direction
        
        for layer in range(1, self.num_layers):
            partial_d = (1/batch_size) * self.adjustments[layer]
            #print(self.weights[layer])
            self.weights[layer][:-1, :] += learning_rate * -partial_d
            self.weights[layer][-1, :] += learning_rate *1e-3* -partial_d[-1, :]
            #print(self.weights[layer])

    def train(self, inputs, targets, num_epochs, learning_rate=0.01, stop_accuracy=1e-3):
        error = []
        for iteration in range(num_epochs):
            for i in range(len(inputs)):
                x = inputs[i]
                y = targets[i]
                # Pass the training set through our neural network
                output = self.forward_propagate(x)

                # Calculate the error
                loss = self.sum_squared_error(output[self.num_layers], y)
                error.append(loss)

                # Calculate Adjustements
                self.__back_propagate(output, y)

            self.__gradient_descente(i, learning_rate)

            # Check if accuarcy criterion is satisfied
            if iteration > 0 and iteration %1000 == 0:
                print( iteration,np.mean(np.abs(error[-(i+1):])))
            if np.mean(np.abs(error[-(i+1):])) < stop_accuracy and iteration > 0:
                print( iteration,np.mean(np.abs(error[-(i+1):])))

        return(np.asarray(error), iteration+1)

In [197]:
def predict_accuracy(data,labels):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(len(data)):
        res = nn.predict(data[i])[0]
        if res[0] >= 0.5 and labels[i][0] == 1:
            TP += 1
        elif res[0] < 0.5 and labels[i][0] == 0:
            TN += 1
        elif res[0] < 0.5 and labels[i][0] == 1:
            FN = FN + 1
        elif res[0] > 0.5 and labels[i][0] == 0:
            FP = FP + 1
    Acc = (TP+TN)/(TP+FP+TN+FN)
    if (TP+FP == 0):
        Prec = 0
    else:
        Prec = TP/(TP+FP)
    Rec = TP/(TP+FN)
    if Prec==0 and Rec == 0:
        F1 = 0
    else:
        F1 = 2*Prec*Rec/(Prec+Rec)
    return Acc,Prec,Rec,F1

In [193]:
dataset = pd.read_csv('dataset.csv')
sample = preprocess(dataset)
Y = sample.Exited.values
X = sample.drop('Exited',axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
Y = y_train.reshape(y_train.shape[0],1)

3


In [194]:
# Create instance of a neural network
nn = NeuralNetwork()
# Add Layers (Input layer is created by default)
nn.add_layer((9, 7))
nn.add_layer((7, 5))
nn.add_layer((5, 1))

In [195]:
training_data = X
training_labels = Y
print(training_labels.shape,training_data.shape)
error, iteration = nn.train(training_data, training_labels,10000,learning_rate=0.005)
print('Error = ', np.mean(np.abs(error[-4:])))
print('Epoches needed to train = ', iteration)

(7200, 1) (7200, 9, 1)
1000 0.0827918120687047
2000 0.0654924890428548
3000 0.06651721575952736
4000 0.0628677745213538
5000 0.0629423944089253
6000 0.06819897205951828
7000 0.06193877345930545
8000 0.06257866936375076
9000 0.06358192910926418
Error =  0.11169834505953691
Epoches needed to train =  10000


In [198]:
Acc,Prec,Rec,F1 = predict_accuracy(training_data,training_labels)
print("Training Accuracy",Acc)
print("Training Precision",Prec)
print("Training Recall",Rec)
print("Training F1",F1)

Training Accuracy 0.8395833333333333
Training Precision 0.6310517529215359
Training Recall 0.5146358066712049
Training F1 0.5669291338582678


In [199]:
X = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
Y = y_test.reshape(y_test.shape[0],1)

In [201]:
testing_data = X
testing_labels = Y
Acc,Prec,Rec,F1 = predict_accuracy(testing_data,testing_labels)
print("Testing Accuracy",Acc)
print("Testing Precision",Prec)
print("Testing Recall",Rec)
print("Testing F1",F1)

Testing Accuracy 0.8261111111111111
Testing Precision 0.5830508474576271
Testing Recall 0.47513812154696133
Testing F1 0.5235920852359208
